In [1]:
import requests
from bs4 import BeautifulSoup

categories = [
    "recettes/index/categorie/entree/",
    "recettes/index/categorie/plat-principal/",
    "recettes/index/categorie/dessert/",
    "recettes/index/categorie/boissons/"
]


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}


all_urls = []

for w in range(2,10,1):
    for elem in categories:
        url = f"https://www.marmiton.org/{elem}{w}"
        r = requests.get(url, headers=headers)
    
        soup = BeautifulSoup(r.content, 'html.parser')
    
        for i in soup.find_all("div", class_="mrtn-card__title"):
            for link in i.find_all("a"):
                recipe_url = link.get("href")
                all_urls.append(recipe_url)


# for urls in all_urls:
#     print(urls)
print(len(all_urls))

960


In [5]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm


dict_final = []
list_benford = []

def webscraping(url):

    """
    Extrait les données d'une recette à partir d'une URL Marmiton.
    
    Args:
        url (str): URL de la recette à extraire
        
    Returns:
        tuple: Un tuple contenant:
            - dict: Dictionnaire avec les détails de la recette (titre, note, temps, ingrédients, etc.)
            - list: Liste des premiers chiffres pour l'analyse de Benford (qui sert juste à vérifier si la loi de benford est respectée)
            
    Raises:
        requests.exceptions.RequestException: Si la requête HTTP échoue
    """

    list_benford_temp = []
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    def extract_elements(soup, selector):
        return [element.get_text(strip=True) for element in soup.select(selector)]
    
    ingredients_nbr = extract_elements(soup, ".card-ingredient-quantity")
    ingredients = extract_elements(soup, ".ingredient-name")
    steps = extract_elements(soup, ".recipe-step-list p")
    equipment = extract_elements(soup, ".card-utensil-quantity")
    title = extract_elements(soup, "h1")[0] if extract_elements(soup, "h1") else "Titre inconnu"
    note = extract_elements(soup, "span.recipe-header__rating-text")[0] if extract_elements(soup, "span.recipe-header__rating-text") else "Note inconnue"

    # Après
    infos = extract_elements(soup, ".recipe-primary__item span")
    temps_preparation = infos[0] if len(infos) > 0 else "Inconnu"
    difficulte = infos[1] if len(infos) > 1 else "Inconnu"
    prix = infos[2] if len(infos) > 2 else "Inconnu"

    
    recipe_data = {
        "titre": title,
        "note": note[:3],
        "temps de préparation": temps_preparation,
        "difficulté":difficulte,
        "prix":prix,
        "ingredients": [],
        "equipement": [],
        "etape": [],     
    }

    for ingredient, ingredient_nbr in zip(ingredients, ingredients_nbr):
        if ingredient_nbr and ingredient_nbr[0].isdigit():
            pass
        else:
            ingredient_nbr = 1
        try:
            list_benford.append(int(ingredient_nbr[0]))
   
        except:
            list_benford.append(int(ingredient_nbr))
    
        recipe_data["ingredients"].append({"quantity": ingredient_nbr, "name": ingredient})
    
    if equipment: 
        lst_equipement=[]
        for elem in equipment:
            # print(equipment)
            lst_equipement.append(' '.join(elem.split()))
        recipe_data["equipement"]=lst_equipement
    # liste_temp=[]
    # for elem in equipment:
    #     x=elem.strip()
    # recipe_data["equipement"]=equipment
    #je ne garde pas l'ajout d'ustensil car il va être essentiellement composé de 1 (on à rarement besoin de 3 mixeurs)
    # for w in range(len(recipe_data["equipement"])):
        # list_benford_temp.append(recipe_data["equipement"][w][0])
        
    for i, step in enumerate(steps, 1):
        recipe_data["etape"].append({"step_number": i, "description": step})
        #je fais en sorte de ne garder que la première décimale
        d=str(i)
        f=d[0]
        list_benford_temp.append(f)
    return recipe_data, list_benford_temp

# urls = [
#     "https://www.marmiton.org/recettes/recette_soupe-de-champignons-ultra-facile_16320.aspx",
# ]


# for url in all_urls:
#     recipe, list_benford_temp = webscraping(url)
#     dict_final.append(recipe)
#     list_benford.extend(list_benford_temp)

for url in tqdm(all_urls, desc="Scraping en cours", unit="recette"):
    recipe, list_benford_temp = webscraping(url)
    dict_final.append(recipe)
    list_benford.extend(list_benford_temp)


# print(dict_final) 
# print(list_benford)
print(len(list_benford))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.74/960 [00:32<06:35,  2.24recette/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.97/960 [01:29<04:49,  2.63recette/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.07/960 [01:36<05:44,  2.19recette/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.32/960 [02:34<04:31,  2.31recette/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.58/960 [04:16<02:52,  2.34recette/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.67/960 [05:04<02:16,  2.15recette/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.01/960 [06:11<01:23,  1.90recette/s]
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.09/960 [06:18<01:31,  1.65recette/s]
Some characters could not be decoded, an

[{'titre': 'Tartare de saumon : la recette inratable', 'note': '4.6', 'temps de préparation': '10 min', 'difficulté': 'très facile', 'prix': 'moyen', 'ingredients': [{'quantity': 1, 'name': 'sel'}, {'quantity': 1, 'name': 'poivre'}, {'quantity': '1filet', 'name': 'saumon frais'}, {'quantity': '1', 'name': 'oignon blanc'}, {'quantity': 1, 'name': "huile d'olive"}, {'quantity': 1, 'name': 'jus de citron'}], 'equipement': ['1 cercle à pâtisserie', '1 Mixeur', '1 couteau', '1 saladiers', '1 Couvercle', '1 dénoyauteur', '1 balance de cuisine'], 'etape': [{'step_number': 1, 'description': "Enlever la peau du filet de saumon, et le passer sous l'eau pour enlever d'éventuelles écailles."}, {'step_number': 2, 'description': "Peler un oignon et le hacher finement au mixer. Couper le filet en six morceaux et les passer au mixer avec les oignons, 2 cuillères à soupe d'huile d'olive, deux pincée de sel, un tour du moulin à poivre, et deux cuillères à café de jus de citron. Ne pas trop mouliner le s

In [ ]:
import pandas as pd
df_csv=pd.DataFrame.from_dict(dict_final)

In [22]:
from pandas import read_csv
df_csv=read_csv("recette.csv")
df_csv['temps de préparation'] = df_csv['temps de préparation'].fillna('inconnu')
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0.2          960 non-null    int64 
 1   Unnamed: 0.1          960 non-null    int64 
 2   Unnamed: 0            960 non-null    int64 
 3   titre                 960 non-null    object
 4   note                  960 non-null    object
 5   temps de préparation  960 non-null    object
 6   difficulté            960 non-null    object
 7   prix                  960 non-null    object
 8   ingredients           960 non-null    object
 9   equipement            960 non-null    object
 10  etape                 960 non-null    object
dtypes: int64(3), object(8)
memory usage: 82.6+ KB


In [23]:


df_csv.to_csv('recette.csv')
# df_csv[:50].to_csv('recette2.csv')
df_csv

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,titre,note,temps de préparation,difficulté,prix,ingredients,equipement,etape
0,0,0,0,Tartare de saumon : la recette inratable,4.6,10 min,très facile,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 cercle à pâtisserie', '1 Mixeur', '1 coute...","[{'step_number': 1, 'description': ""Enlever la..."
1,1,1,1,Terrine de poisson au basilic,4.6,55 min,facile,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 robot pâtissier', '1 Mixeur', '1 saladiers...","[{'step_number': 1, 'description': 'Mixer au r..."
2,2,2,2,Oeufs en Meurette de ma Maman,4.8,1 h,moyenne,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 Mixeur', '1 couteau', '1 Set 3 poêles', '1...","[{'step_number': 1, 'description': ""Couper le ..."
3,3,3,3,Soupe froide de courgettes à la menthe et chèv...,4.3,50 min,très facile,bon marché,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 louche', '1 mixeur plongeant', '1 Blender ...","[{'step_number': 1, 'description': ""Laver et c..."
4,4,4,4,Pâté aux pommes de terre (Bourbonnais),4.7,1h25,moyenne,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 Cuillère en bois', '1 Couteau', '1 économe...","[{'step_number': 1, 'description': ""Eplucher l..."
...,...,...,...,...,...,...,...,...,...,...,...
955,955,955,955,"Glaçons aromatisés: framboises, fraises, myrti...",0/5,10 min,très facile,bon marché,"[{'quantity': 1, 'name': 'fraises'}, {'quantit...",['1 congélateur'],"[{'step_number': 1, 'description': 'Laver et c..."
956,956,956,956,Limonade au pamplemousse et grenadine,5/5,1 min,très facile,bon marché,"[{'quantity': '50cl', 'name': 'pamplemousse'},...",[],"[{'step_number': 1, 'description': 'Dans une b..."
957,957,957,957,"Smoothie Violet (Banane, Cassis et Soja)",4.8,5 min,très facile,bon marché,"[{'quantity': '1pincée', 'name': 'sucre'}, {'q...","['1 blender', '1 Mixeur']","[{'step_number': 1, 'description': 'Mixer le t..."
958,958,958,958,Vin d'oranges amères rouge,3/5,5 min,très facile,moyen,"[{'quantity': '25g', 'name': ""écorce d'orange""...","['1 Entonnoir', '1 balance de cuisine']","[{'step_number': 1, 'description': 'Faites mac..."


![image](Screenshot_231.png)

DROP DATABASE IF EXISTS projet;

CREATE DATABASE projet;
USE projet;

CREATE TABLE `recette` (
    `id` INTEGER NOT NULL AUTO_INCREMENT,
    `note` DECIMAL NOT NULL,
    `temps_preparation` VARCHAR(255) NOT NULL,
    `difficulte` VARCHAR(255) NOT NULL,
    `prix` VARCHAR(255) NOT NULL,
    `titre` VARCHAR(255) DEFAULT 'inconnu',
    PRIMARY KEY(`id`)
);

CREATE TABLE `ingredient` (
    `id` INTEGER NOT NULL AUTO_INCREMENT,
    `nom` VARCHAR(255) DEFAULT 'inconnu' NULL,
    `quantite` VARCHAR(255) NOT NULL,
    PRIMARY KEY(`id`)
);

CREATE TABLE `etape` (
	`id` INTEGER NOT NULL AUTO_INCREMENT,
	`numero` INTEGER NOT NULL,
	`etape` TEXT NOT NULL,
	`recette_id` INTEGER NOT NULL,
	PRIMARY KEY(`id`, `recette_id`)
);


CREATE TABLE `materiel` (
    `id` INTEGER NOT NULL AUTO_INCREMENT,
    `equipement` VARCHAR(255) DEFAULT 'inconnu',
    PRIMARY KEY(`id`)
);

CREATE TABLE `recette_ingredient` (
    `recette_id` INTEGER NOT NULL,
    `ingredient_id` INTEGER NOT NULL,
    PRIMARY KEY(`recette_id`, `ingredient_id`)
);

CREATE TABLE `recette_materiel` (
    `recette_id` INTEGER NOT NULL,
    `materiel_id` INTEGER NOT NULL,
    PRIMARY KEY(`recette_id`, `materiel_id`)
);

ALTER TABLE `recette_ingredient`
ADD FOREIGN KEY(`recette_id`) REFERENCES `recette`(`id`)
ON UPDATE CASCADE ON DELETE SET CASCADE;
ALTER TABLE `recette_ingredient`
ADD FOREIGN KEY(`ingredient_id`) REFERENCES `ingredient`(`id`)
ON UPDATE CASCADE ON DELETE SET CASCADE;
ALTER TABLE `etape`
ADD FOREIGN KEY(`recette_id`) REFERENCES `recette`(`id`)
ON UPDATE CASCADE ON DELETE SET CASCADE;
ALTER TABLE `recette_materiel`
ADD FOREIGN KEY(`recette_id`) REFERENCES `recette`(`id`)
ON UPDATE CASCADE ON DELETE SET CASCADE;
ALTER TABLE `recette_materiel`
ADD FOREIGN KEY(`materiel_id`) REFERENCES `materiel`(`id`)
ON UPDATE CASCADE ON DELETE SET CASCADE;

In [38]:
import pandas as pd
import numpy as np
password="Arthur-2002"
df = pd.read_csv("recette.csv")

df.rename(columns={
    "titre": "titre",
    "note": "note",
    "temps de préparation": "temps_preparation",
    "difficulté": "difficulte",
    "prix": "prix",
    "ingredients": "ingredient",
    "equipement": "materiel",
    "etape": "etape"
}, inplace=True)

df["note"] = pd.to_numeric(df["note"], errors="coerce").fillna(0).round(1)
df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,titre,note,temps_preparation,difficulte,prix,ingredient,materiel,etape
0,0,0,0,0,Tartare de saumon : la recette inratable,4.6,10 min,très facile,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 cercle à pâtisserie', '1 Mixeur', '1 coute...","[{'step_number': 1, 'description': ""Enlever la..."
1,1,1,1,1,Terrine de poisson au basilic,4.6,55 min,facile,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 robot pâtissier', '1 Mixeur', '1 saladiers...","[{'step_number': 1, 'description': 'Mixer au r..."
2,2,2,2,2,Oeufs en Meurette de ma Maman,4.8,1 h,moyenne,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 Mixeur', '1 couteau', '1 Set 3 poêles', '1...","[{'step_number': 1, 'description': ""Couper le ..."
3,3,3,3,3,Soupe froide de courgettes à la menthe et chèv...,4.3,50 min,très facile,bon marché,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 louche', '1 mixeur plongeant', '1 Blender ...","[{'step_number': 1, 'description': ""Laver et c..."
4,4,4,4,4,Pâté aux pommes de terre (Bourbonnais),4.7,1h25,moyenne,moyen,"[{'quantity': 1, 'name': 'sel'}, {'quantity': ...","['1 Cuillère en bois', '1 Couteau', '1 économe...","[{'step_number': 1, 'description': ""Eplucher l..."
...,...,...,...,...,...,...,...,...,...,...,...,...
955,955,955,955,955,"Glaçons aromatisés: framboises, fraises, myrti...",0.0,10 min,très facile,bon marché,"[{'quantity': 1, 'name': 'fraises'}, {'quantit...",['1 congélateur'],"[{'step_number': 1, 'description': 'Laver et c..."
956,956,956,956,956,Limonade au pamplemousse et grenadine,0.0,1 min,très facile,bon marché,"[{'quantity': '50cl', 'name': 'pamplemousse'},...",[],"[{'step_number': 1, 'description': 'Dans une b..."
957,957,957,957,957,"Smoothie Violet (Banane, Cassis et Soja)",4.8,5 min,très facile,bon marché,"[{'quantity': '1pincée', 'name': 'sucre'}, {'q...","['1 blender', '1 Mixeur']","[{'step_number': 1, 'description': 'Mixer le t..."
958,958,958,958,958,Vin d'oranges amères rouge,0.0,5 min,très facile,moyen,"[{'quantity': '25g', 'name': ""écorce d'orange""...","['1 Entonnoir', '1 balance de cuisine']","[{'step_number': 1, 'description': 'Faites mac..."


In [39]:
import ast
test=df.ingredient.iloc[1]

def eval(evaluated):
     """
    Évalue une chaîne de caractères représentant une liste d'ingrédients en structure Python.
    
    Args:
        evaluated (str): Chaîne de caractères représentant une liste d'ingrédients
        
    Returns:
        list: Liste de tuples (quantité, nom) pour chaque ingrédient
        
    Note:
        Utilise ast.literal_eval pour une évaluation sécurisée de la chaîne
    """
    #https://www.geeksforgeeks.org/difference-between-eval-and-ast-literal-eval-in-python/
    ingredients_list = ast.literal_eval(evaluated)
    extracted_ingredients = [(item["quantity"], item["name"]) for item in ingredients_list]
    return extracted_ingredients
result=eval(test)
print(result)

[(1, 'sel'), (1, 'poivre'), ('2cuillères à soupe', 'crème fraîche'), ('400g', 'saumon'), ('300g', 'poisson'), ('1petite boîte', 'concentré de tomates'), ('1petit bouquet', 'basilic'), ('2', 'oeufs')]


In [40]:
lst1,lst2=[],[]
for i in range(len(result)):
    lst1.append(result[i][0])
    lst2.append(result[i][1])
print(lst2)  
print(lst1)

['sel', 'poivre', 'crème fraîche', 'saumon', 'poisson', 'concentré de tomates', 'basilic', 'oeufs']
[1, 1, '2cuillères à soupe', '400g', '300g', '1petite boîte', '1petit bouquet', '2']


In [41]:
def separation(evaluated):
    """
    Sépare les quantités et les noms des ingrédients à partir d'une chaîne évaluée.
    
    Args:
        evaluated (str): Chaîne de caractères représentant une liste d'ingrédients
        
    Returns:
        tuple: Un tuple contenant:
            - list: Liste des quantités d'ingrédients
            - list: Liste des noms d'ingrédients
    """
    def eval(evaluated):
    #https://www.geeksforgeeks.org/difference-between-eval-and-ast-literal-eval-in-python/
        ingredients_list = ast.literal_eval(evaluated)
        extracted_ingredients = [(item["quantity"], item["name"]) for item in ingredients_list]
        return extracted_ingredients
    result=eval(evaluated)
    lst1,lst2=[],[]
    for i in range(len(result)):
        lst1.append(result[i][0])
        lst2.append(result[i][1])
    return lst1,lst2
result=separation(df.ingredient.iloc[1])
result[0]
print((separation(df.ingredient.iloc[1]))[0])

[1, 1, '2cuillères à soupe', '400g', '300g', '1petite boîte', '1petit bouquet', '2']


In [42]:
df_ingredients = df[["titre", "ingredient"]].copy()
df_ingredients['quantite'] = df_ingredients.ingredient.map(lambda x:(separation(x))[0])
df_ingredients['nom'] = df_ingredients['ingredient'].map(lambda x:(separation(x))[1])
df_temp = df_ingredients[["titre", "nom"]]
df_ingredients.drop(['ingredient',"nom"], axis=1, inplace=True)
df_ingredients = df_ingredients.explode("quantite").reset_index(drop=True)
df_temp=df_temp.explode("nom").reset_index(drop=True)
df_ingredients["nom"]=df_temp["nom"]
df_ingredients

,titre,quantite,nom
0,Tartare de saumon : la recette inratable,1,sel
1,Tartare de saumon : la recette inratable,1,poivre
2,Tartare de saumon : la recette inratable,1filet,saumon frais
3,Tartare de saumon : la recette inratable,1,oignon blanc
4,Tartare de saumon : la recette inratable,1,huile d'olive
...,...,...,...
7092,Cocktail au chocolat,1cuillère à café,sucre glace
7093,Cocktail au chocolat,0.5cuillère à soupe,crème fraîche
7094,Cocktail au chocolat,10cl,lait
7095,Cocktail au chocolat,10cl,crème


In [43]:
# Normalisation des équipements (une ligne par équipement)
df_materiel = df[["titre", "materiel"]].copy()
df_materiel = df_materiel.dropna()
df_materiel = df_materiel.assign(materiel=df_materiel["materiel"].str.split(","))
df_materiel = df_materiel.explode("materiel").reset_index(drop=True)
df_materiel


,titre,materiel
0,Tartare de saumon : la recette inratable,['1 cercle à pâtisserie'
1,Tartare de saumon : la recette inratable,'1 Mixeur'
2,Tartare de saumon : la recette inratable,'1 couteau'
3,Tartare de saumon : la recette inratable,'1 saladiers'
4,Tartare de saumon : la recette inratable,'1 Couvercle'
...,...,...
5053,"Smoothie Violet (Banane, Cassis et Soja)",'1 Mixeur']
5054,Vin d'oranges amères rouge,['1 Entonnoir'
5055,Vin d'oranges amères rouge,'1 balance de cuisine']
5056,Cocktail au chocolat,['1 Presse-ail'


In [44]:
def separation1(evaluated):
    """
    Sépare les numéros d'étape et les descriptions à partir d'une chaîne évaluée.
    
    Args:
        evaluated (str): Chaîne de caractères représentant une liste d'étapes de recette
        
    Returns:
        tuple: Un tuple contenant:
            - list: Liste des numéros d'étape
            - list: Liste des descriptions d'étape
    """
    def eval(evaluated):
    #https://www.geeksforgeeks.org/difference-between-eval-and-ast-literal-eval-in-python/
        ingredients_list = ast.literal_eval(evaluated)
        extracted_ingredients = [(item["step_number"], item["description"]) for item in ingredients_list]
        return extracted_ingredients
    result=eval(evaluated)
    # print(result)
    lst1,lst2=[],[]
    for i in range(len(result)):
        lst1.append(result[i][0])
        lst2.append(result[i][1])
        # print(lst2[i])
    return lst1,lst2

df_etapes = df[["titre", "etape"]].copy()
df_etapes = df_etapes.dropna()
df_temps=pd.DataFrame()
df_temps['titre']=df_etapes.titre
df_temps['numero'] = df_etapes.etape.map(lambda x:(separation1(x))[0])
df_temps['description'] = df_etapes.etape.map(lambda x:(separation1(x))[1])
df_temps

,titre,numero,description
0,Tartare de saumon : la recette inratable,"[1, 2, 3]","[Enlever la peau du filet de saumon, et le pas..."
1,Terrine de poisson au basilic,"[1, 2, 3, 4, 5, 6, 7, 8]","[Mixer au robot les filets de poisson crus., L..."
2,Oeufs en Meurette de ma Maman,"[1, 2, 3, 4, 5, 6, 7, 8]","[Couper le lard en dés, émincer l'oignon et le..."
3,Soupe froide de courgettes à la menthe et chèv...,"[1, 2, 3, 4]",[Laver et couper en morceaux grossiers les cou...
4,Pâté aux pommes de terre (Bourbonnais),"[1, 2, 3, 4, 5, 6, 7]","[Eplucher les pommes de terre, les laver, les ..."
...,...,...,...
955,"Glaçons aromatisés: framboises, fraises, myrti...","[1, 2, 3, 4]",[Laver et couper les fruits et les herbes arom...
956,Limonade au pamplemousse et grenadine,"[1, 2]","[Dans une bouteille ou dans un pichet, verse l..."
957,"Smoothie Violet (Banane, Cassis et Soja)","[1, 2]","[Mixer le tout environ 30 secondes !, C'est la..."
958,Vin d'oranges amères rouge,"[1, 2, 3, 4]","[Faites macérer vin, camomille, orange et sucr..."


In [45]:
df_etapes = df_temps.explode(["numero", "description"]).reset_index(drop=True)
df_etapes

,titre,numero,description
0,Tartare de saumon : la recette inratable,1,"Enlever la peau du filet de saumon, et le pass..."
1,Tartare de saumon : la recette inratable,2,Peler un oignon et le hacher finement au mixer...
2,Tartare de saumon : la recette inratable,3,"Placer la préparation dans un saladier, couvri..."
3,Terrine de poisson au basilic,1,Mixer au robot les filets de poisson crus.
4,Terrine de poisson au basilic,2,Les verser dans un saladier puis incorporer un...
...,...,...,...
5854,Vin d'oranges amères rouge,1,"Faites macérer vin, camomille, orange et sucre..."
5855,Vin d'oranges amères rouge,2,Filtrez puis ajoutez l'eau de vie et mettre en...
5856,Vin d'oranges amères rouge,3,"Si mis en carafe, dégustez aussitôt."
5857,Vin d'oranges amères rouge,4,"Si mis dans une bouteille bien fermée, se gard..."


In [46]:
df.drop(['ingredient', 'materiel', 'etape','Unnamed: 0.1','Unnamed: 0'], axis=1, inplace=True)
df

,Unnamed: 0.3,Unnamed: 0.2,titre,note,temps_preparation,difficulte,prix
0,0,0,Tartare de saumon : la recette inratable,4.6,10 min,très facile,moyen
1,1,1,Terrine de poisson au basilic,4.6,55 min,facile,moyen
2,2,2,Oeufs en Meurette de ma Maman,4.8,1 h,moyenne,moyen
3,3,3,Soupe froide de courgettes à la menthe et chèv...,4.3,50 min,très facile,bon marché
4,4,4,Pâté aux pommes de terre (Bourbonnais),4.7,1h25,moyenne,moyen
...,...,...,...,...,...,...,...
955,955,955,"Glaçons aromatisés: framboises, fraises, myrti...",0.0,10 min,très facile,bon marché
956,956,956,Limonade au pamplemousse et grenadine,0.0,1 min,très facile,bon marché
957,957,957,"Smoothie Violet (Banane, Cassis et Soja)",4.8,5 min,très facile,bon marché
958,958,958,Vin d'oranges amères rouge,0.0,5 min,très facile,moyen


In [47]:
# Affichage des DataFrames transformés
print("Recettes :\n", df.head())
print("\nIngrédients :\n", df_ingredients.head())
print("\nMatériel :\n", df_materiel.head())
print("\nÉtapes :\n", df_etapes.head())

Recettes :
    Unnamed: 0.3  Unnamed: 0.2  \
0             0             0   
1             1             1   
2             2             2   
3             3             3   
4             4             4   

                                               titre  note temps_preparation  \
0           Tartare de saumon : la recette inratable   4.6            10 min   
1                      Terrine de poisson au basilic   4.6            55 min   
2                      Oeufs en Meurette de ma Maman   4.8               1 h   
3  Soupe froide de courgettes à la menthe et chèv...   4.3            50 min   
4             Pâté aux pommes de terre (Bourbonnais)   4.7              1h25   

    difficulte        prix  
0  très facile       moyen  
1       facile       moyen  
2      moyenne       moyen  
3  très facile  bon marché  
4      moyenne       moyen  

Ingrédients :
                                       titre quantite            nom
0  Tartare de saumon : la recette inratable       

![image](Screenshot_231.png)

In [54]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Arthur-2002",
  database="projet"
)

mycursor = mydb.cursor()


# Désactiver temporairement les contraintes de clés étrangères
mycursor.execute("SET FOREIGN_KEY_CHECKS = 0")

# Supprimer toutes les données des tables
cleanup_queries = [
    "TRUNCATE TABLE recette_ingredient",
    "TRUNCATE TABLE recette_materiel",
    "TRUNCATE TABLE etape",
    "TRUNCATE TABLE ingredient",
    "TRUNCATE TABLE materiel",
    "TRUNCATE TABLE recette",
]

for query in cleanup_queries:
    mycursor.execute(query)

# Réinitialiser les auto-increment
reset_queries = [
    "ALTER TABLE recette AUTO_INCREMENT = 1",
    "ALTER TABLE ingredient AUTO_INCREMENT = 1",
    "ALTER TABLE materiel AUTO_INCREMENT = 1",
    "ALTER TABLE etape AUTO_INCREMENT = 1"
]

for query in reset_queries:
    mycursor.execute(query)

# Réactiver les contraintes de clés étrangères
mycursor.execute("SET FOREIGN_KEY_CHECKS = 1")

mydb.commit()
print("Database cleaned successfully!")



Database cleaned successfully!


In [55]:
for index, row in df.iterrows():
    #On insère les données du dataframe dans la database
    sql = "INSERT INTO recette (titre, note, temps_preparation, difficulte, prix) VALUES (%s, %s, %s, %s, %s)"  
    values = (row['titre'], row['note'], row['temps_preparation'], row['difficulte'], row['prix'])
    #On exécute la requête
    mycursor.execute(sql, values)
    #permet de récupérer l'id de la dernière ligne  (donc de tout les id vu que c'est une boucle)
    recipe_id = mycursor.lastrowid  
    
    #On itère sur chaque ligne du DataFrame df_ingredients où le titre correspond à celui de la recette actuelle
    for _, ing_row in df_ingredients[df_ingredients['titre'] == row['titre']].iterrows():
        sql = "INSERT IGNORE INTO ingredient (nom, quantite) VALUES (%s, %s)"
        values = (ing_row['nom'], ing_row['quantite'])
        mycursor.execute(sql, values)
        ingredient_id = mycursor.lastrowid
        
        #On insert lss relations entre les id des recettes et les id des ingrédients dans la table recette_ingredient
        sql = "INSERT INTO recette_ingredient (recette_id, ingredient_id) VALUES (%s, %s)"
        values = (recipe_id, ingredient_id)
        mycursor.execute(sql, values)
    
    for _, step_row in df_etapes[df_etapes['titre'] == row['titre']].iterrows():
        sql = "INSERT INTO etape (numero, etape, recette_id) VALUES (%s, %s, %s)"
        values = (step_row['numero'], step_row['description'], recipe_id)
        mycursor.execute(sql, values)
    
    for _, mat_row in df_materiel[df_materiel['titre'] == row['titre']].iterrows():
        sql = "INSERT IGNORE INTO materiel (equipement) VALUES (%s)"
        values = (mat_row['materiel'],)
        mycursor.execute(sql, values)
        materiel_id = mycursor.lastrowid
        
        sql = "INSERT INTO recette_materiel (recette_id, materiel_id) VALUES (%s, %s)"
        values = (recipe_id, materiel_id)
        mycursor.execute(sql, values)

#on envois les requetes et on ferme la connection
mydb.commit()
mydb.close()

In [ ]:
import mysql.connector
import pandas as pd

# Connexion à la base de données
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Arthur-2002",
  database="projet"
)

mycursor = mydb.cursor(dictionary=True)

# Fonction pour récupérer toutes les recettes liées à un ingrédient
def get_recipes_by_ingredient(ingredient_id=None, ingredient_name=None):
    """
    Récupère toutes les recettes qui utilisent un ingrédient spécifique.
    
    Args:
        ingredient_id (int, optional): ID de l'ingrédient
        ingredient_name (str, optional): Nom de l'ingrédient
        
    Returns:
        DataFrame: Recettes qui utilisent cet ingrédient
    """
    if ingredient_id is None and ingredient_name is None:
        raise ValueError("Vous devez spécifier soit l'ID soit le nom de l'ingrédient")
    
    if ingredient_name and not ingredient_id:
        # Récupérer l'ID de l'ingrédient par son nom
        mycursor.execute("SELECT id FROM ingredient WHERE nom = %s", (ingredient_name,))
        result = mycursor.fetchone()
        if not result:
            print(f"Aucun ingrédient trouvé avec le nom '{ingredient_name}'")
            return pd.DataFrame()
        ingredient_id = result['id']
    
    # Récupérer les recettes
    query = """
    SELECT r.id, r.titre, r.note, r.temps_preparation, r.difficulte, r.prix
    FROM recette r
    JOIN recette_ingredient ri ON r.id = ri.recette_id
    WHERE ri.ingredient_id = %s
    """
    mycursor.execute(query, (ingredient_id,))
    recipes = mycursor.fetchall()
    
    if not recipes:
        print(f"Aucune recette n'utilise l'ingrédient avec l'ID {ingredient_id}")
        return pd.DataFrame()
    
    return pd.DataFrame(recipes)

# Fonction pour récupérer tous les ingrédients d'une recette
def get_ingredients_by_recipe(recipe_id=None, recipe_name=None):
    """
    Récupère tous les ingrédients utilisés dans une recette spécifique.
    
    Args:
        recipe_id (int, optional): ID de la recette
        recipe_name (str, optional): Titre de la recette
        
    Returns:
        DataFrame: Ingrédients utilisés dans cette recette
    """
    if recipe_id is None and recipe_name is None:
        raise ValueError("Vous devez spécifier soit l'ID soit le nom de la recette")
    
    if recipe_name and not recipe_id:
        # Récupérer l'ID de la recette par son titre
        mycursor.execute("SELECT id FROM recette WHERE titre = %s", (recipe_name,))
        result = mycursor.fetchone()
        if not result:
            print(f"Aucune recette trouvée avec le titre '{recipe_name}'")
            return pd.DataFrame()
        recipe_id = result['id']
    
    # Récupérer les ingrédients
    query = """
    SELECT i.id, i.nom, i.quantite
    FROM ingredient i
    JOIN recette_ingredient ri ON i.id = ri.ingredient_id
    WHERE ri.recette_id = %s
    """
    mycursor.execute(query, (recipe_id,))
    ingredients = mycursor.fetchall()
    
    if not ingredients:
        print(f"Aucun ingrédient trouvé pour la recette avec l'ID {recipe_id}")
        return pd.DataFrame()
    
    return pd.DataFrame(ingredients)

# Fonction pour récupérer tout le matériel d'une recette
def get_equipment_by_recipe(recipe_id=None, recipe_name=None):
    """
    Récupère tout le matériel utilisé dans une recette spécifique.
    
    Args:
        recipe_id (int, optional): ID de la recette
        recipe_name (str, optional): Titre de la recette
        
    Returns:
        DataFrame: Matériel utilisé dans cette recette
    """
    if recipe_id is None and recipe_name is None:
        raise ValueError("Vous devez spécifier soit l'ID soit le nom de la recette")
    
    if recipe_name and not recipe_id:
        # Récupérer l'ID de la recette par son titre
        mycursor.execute("SELECT id FROM recette WHERE titre = %s", (recipe_name,))
        result = mycursor.fetchone()
        if not result:
            print(f"Aucune recette trouvée avec le titre '{recipe_name}'")
            return pd.DataFrame()
        recipe_id = result['id']
    
    # Récupérer le matériel
    query = """
    SELECT m.id, m.equipement
    FROM materiel m
    JOIN recette_materiel rm ON m.id = rm.materiel_id
    WHERE rm.recette_id = %s
    """
    mycursor.execute(query, (recipe_id,))
    equipment = mycursor.fetchall()
    
    if not equipment:
        print(f"Aucun matériel trouvé pour la recette avec l'ID {recipe_id}")
        return pd.DataFrame()
    
    return pd.DataFrame(equipment)

# Fonction pour récupérer toutes les recettes qui utilisent un matériel spécifique
def get_recipes_by_equipment(equipment_id=None, equipment_name=None):
    """
    Récupère toutes les recettes qui utilisent un matériel spécifique.
    
    Args:
        equipment_id (int, optional): ID du matériel
        equipment_name (str, optional): Nom du matériel
        
    Returns:
        DataFrame: Recettes qui utilisent ce matériel
    """
    if equipment_id is None and equipment_name is None:
        raise ValueError("Vous devez spécifier soit l'ID soit le nom du matériel")
    
    if equipment_name and not equipment_id:
        # Récupérer l'ID du matériel par son nom
        mycursor.execute("SELECT id FROM materiel WHERE equipement LIKE %s", (f"%{equipment_name}%",))
        result = mycursor.fetchone()
        if not result:
            print(f"Aucun matériel trouvé avec le nom '{equipment_name}'")
            return pd.DataFrame()
        equipment_id = result['id']
    
    # Récupérer les recettes
    query = """
    SELECT r.id, r.titre, r.note, r.temps_preparation, r.difficulte, r.prix
    FROM recette r
    JOIN recette_materiel rm ON r.id = rm.recette_id
    WHERE rm.materiel_id = %s
    """
    mycursor.execute(query, (equipment_id,))
    recipes = mycursor.fetchall()
    
    if not recipes:
        print(f"Aucune recette n'utilise le matériel avec l'ID {equipment_id}")
        return pd.DataFrame()
    
    return pd.DataFrame(recipes)

# Fonction pour récupérer les étapes d'une recette
def get_steps_by_recipe(recipe_id=None, recipe_name=None):
    """
    Récupère toutes les étapes d'une recette spécifique.
    
    Args:
        recipe_id (int, optional): ID de la recette
        recipe_name (str, optional): Titre de la recette
        
    Returns:
        DataFrame: Étapes de la recette
    """
    if recipe_id is None and recipe_name is None:
        raise ValueError("Vous devez spécifier soit l'ID soit le nom de la recette")
    
    if recipe_name and not recipe_id:
        # Récupérer l'ID de la recette par son titre
        mycursor.execute("SELECT id FROM recette WHERE titre = %s", (recipe_name,))
        result = mycursor.fetchone()
        if not result:
            print(f"Aucune recette trouvée avec le titre '{recipe_name}'")
            return pd.DataFrame()
        recipe_id = result['id']
    
    # Récupérer les étapes
    query = """
    SELECT id, numero, etape
    FROM etape
    WHERE recette_id = %s
    ORDER BY numero
    """
    mycursor.execute(query, (recipe_id,))
    steps = mycursor.fetchall()
    
    if not steps:
        print(f"Aucune étape trouvée pour la recette avec l'ID {recipe_id}")
        return pd.DataFrame()
    
    return pd.DataFrame(steps)

# Exemples d'utilisation
print("Exemple 1: Trouver toutes les recettes qui utilisent l'ingrédient avec l'ID 12")
recipes_with_ingredient_12 = get_recipes_by_ingredient(ingredient_id=12)
display(recipes_with_ingredient_12)

print("\nExemple 2: Trouver tous les ingrédients de la recette 'Tarte aux pommes'")
ingredients_in_tarte = get_ingredients_by_recipe(recipe_name="Tarte aux pommes")
display(ingredients_in_tarte)

print("\nExemple 3: Trouver toutes les recettes qui utilisent un mixeur")
recipes_with_mixer = get_recipes_by_equipment(equipment_name="mixeur")
display(recipes_with_mixer)

print("\nExemple 4: Trouver les étapes de la première recette")
if len(recipes_with_ingredient_12) > 0:
    first_recipe_id = recipes_with_ingredient_12.iloc[0]['id']
    steps_of_first_recipe = get_steps_by_recipe(recipe_id=first_recipe_id)
    display(steps_of_first_recipe)

In [ ]:
#pour récupérer le premier nombre de chaque quantité (loi benford)
SELECT id, nom, quantite, LEFT(quantite, 1) as first_digit
FROM ingredient
WHERE quantite REGEXP '^[0-9]' LIMIT 10;

In [56]:
pip install jinja2

Note: you may need to restart the kernel to use updated packages.


In [1]:
curl -X POST "http://localhost:8000/sql/recipe/insert?nom=Tarte&description=Une%20tarte%20aux%20pommes&temps_preparation=30&temps_cuisson=45"

SyntaxError: invalid syntax (813853991.py, line 1)

In [4]:
from typing import Optional

class RecipeAPI:
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url

    def insert_recipe(self, 
                     titre: str,
                     note: float,
                     temps_preparation: str,
                     difficulte: str,
                     prix: str) -> Optional[int]:
        try:
            params = {
                "titre": titre,
                "note": note,
                "temps_preparation": temps_preparation,
                "difficulte": difficulte,
                "prix": prix
            }
            response = requests.post(f"{self.base_url}/sql/recipe/insert", params=params)
            response.raise_for_status()  # Lève une exception si status code >= 400
            result = response.json()
            print("Recette ajoutée avec succès:", result)
            return result.get('id')
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de l'insertion: {e}")
            return None

    def update_recipe(self, recipe_id: int, **kwargs) -> bool:
        try:
            response = requests.put(f"{self.base_url}/sql/recipe/{recipe_id}", params=kwargs)
            response.raise_for_status()
            print("Recette mise à jour avec succès:", response.json())
            return True
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la mise à jour: {e}")
            return False

    def delete_recipe(self, recipe_id: int) -> bool:
        try:
            response = requests.delete(f"{self.base_url}/sql/recipe/{recipe_id}")
            response.raise_for_status()
            print("Recette supprimée avec succès:", response.json())
            return True
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la suppression: {e}")
            return False

# Utilisation
api = RecipeAPI()

In [5]:
# Test complet
recipe_id = api.insert_recipe(
    titre="Tarte aux pommes",
    note=4.5,
    temps_preparation="30 min",
    difficulte="facile",
    prix="bon marché"
)

Erreur lors de l'insertion: 422 Client Error: Unprocessable Entity for url: http://localhost:8000/sql/recipe/insert?titre=Tarte+aux+pommes&note=4.5&temps_preparation=30+min&difficulte=facile&prix=bon+march%C3%A9


In [10]:
import requests

# D'abord faire l'insertion
response_insert = requests.post(
    "http://localhost:8000/sql/recipe/insert",
    params={
        "titre": "Tarte aux pommes",
        "note": 4.5,
        "temps_preparation": "30 min",
        "difficulte": "facile",
        "prix": "bon marché"
    }
)

# Récupérer l'ID de la recette insérée
recipe_id = response_insert.json().get('id')

# Ensuite, récupérer la recette pour vérifier
response_get = requests.get(f"http://localhost:8000/sql/recipe/{recipe_id}")
print("\nRécupération de la recette insérée:")
print(response_get.json())


Récupération de la recette insérée:
{'detail': [{'type': 'int_parsing', 'loc': ['path', 'id'], 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'None'}]}
